In [25]:
# STEP 1: Install packages (uncomment if needed)
!pip install transformers gradio torch accelerate gTTs -q

In [26]:
# STEP 2: Import libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import gradio as gr
import torch
from gtts import gTTS
import os
import json
from datetime import datetime, date

In [27]:
# Persistent Task Storage
TASKS_FILE = "tasks.json"

def load_tasks():
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, "r") as f:
            return json.load(f)
    return []

def save_tasks(tasks):
    with open(TASKS_FILE, "w") as f:
        json.dump(tasks, f, indent=2)

def handle_productivity_commands(user_input):
    tasks = load_tasks()
    text = user_input.lower()

    # ADD TASK
    if text.startswith("add task"):
        try:
            parts = user_input.split("due")
            title = parts[0].replace("add task", "").strip()
            due_date = datetime.strptime(parts[1].strip(), "%Y-%m-%d").date()

            tasks.append({
                "title": title,
                "due": due_date.isoformat()
            })
            save_tasks(tasks)
            return f"Task added: {title}, due {due_date}"

        except Exception:
            return "Format error. Try: add task COMP-2540 lab due 2026-01-20"

    # DELETE TASK
    if text.startswith("delete task"):
        title_to_delete = user_input.replace("delete task", "").strip()
        tasks_before = len(tasks)
        tasks = [t for t in tasks if t["title"].lower() != title_to_delete.lower()]

        if len(tasks) < tasks_before:
            save_tasks(tasks)
            return f"Task deleted: {title_to_delete}"
        else:
            return f"No task found with title: {title_to_delete}"

    # WHAT'S DUE
    if "what's due" in text or "whats due" in text:
        if not tasks:
            return "You have no tasks due!"

        today = date.today()
        reply = "Upcoming Tasks:\n"
        for t in tasks:
            due = date.fromisoformat(t["due"])
            days_left = (due - today).days
            reply += f"- {t['title']} (due in {days_left} days)\n"
        return reply

    # DAILY SUMMARY
    if "daily summary" in text:
        today = date.today()
        due_soon = [
            t for t in tasks
            if (date.fromisoformat(t["due"]) - today).days <= 3
        ]

        if not due_soon:
            return "No urgent deadlines in the next 3 days."

        reply = "Deadlines coming up:\n"
        for t in due_soon:
            due = date.fromisoformat(t["due"])
            reply += f"- {t['title']} (due {due})\n"
        return reply

    return None

In [28]:
# STEP 3: Load a SMARTER conversational AI model
print("Loading your AI assistant... (this might take a minute)")

model_name = "google/flan-t5-large"   # Hint: flan-t5-large
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print("Assistant loaded! Ready to chat.")

Loading your AI assistant... (this might take a minute)
Assistant loaded! Ready to chat.


In [29]:
# STEP 4: Create the chat function with better prompting
def chat_with_assistant(user_input, history):

    context = "You are JARVIS, an intelligent and helpful AI assistant. Be conversational and friendly.\n\n"

    if history:
        for human, assistant in history[-3:]:
            context += f"Human: {human}\nJARVIS: {assistant}\n"

    context += f"Human: {user_input}\nJARVIS:"

    inputs = tokenizer(context, return_tensors="pt", truncation=True, max_length=1024)
    outputs = model.generate(
        **inputs,
        max_length=256,     # Longer responses
        num_beams=4,      # Better quality
        temperature=0.7,    # More creative
        do_sample=True,
        top_p=0.9
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [30]:
# STEP 5: Create Gradio interface with voice
def gradio_chat_with_voice(message, history):
    command_response = handle_productivity_commands(message)

    if command_response:
        response = command_response
    else:
        response = chat_with_assistant(message, history)

    try:
        tts = gTTS(text=response, lang="en", slow=False)
        audio_file = "response.mp3"
        tts.save(audio_file)
        return response, audio_file
    except Exception as e:
        print(f"TTS Error: {e}")
        return response, None

In [ ]:
# STEP 6: Create beautiful Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🤖 Your Personal JARVIS Assistant
    ### Built in Google Colab with AI superpowers!
    Ask me anything - I'll respond with text and voice.
    """)

    with gr.Row():
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=500, bubble_full_width=False)

            with gr.Row():
                msg = gr.Textbox(
                    label="Your message",
                    placeholder="Ask me anything...",
                    scale=4
                )
                submit = gr.Button("Send 🚀", scale=1, variant="primary")

            clear = gr.Button("Clear Chat 🗑️")

        with gr.Column(scale=1):
            audio_output = gr.Audio(label="🔊 Voice Response", autoplay=True)

            gr.Markdown("### Quick Examples:")
            example_btns = [
                gr.Button("➕ Add a task"),
                gr.Button("🗑️ Delete a task"),
                gr.Button("📌 Check what's due"),
                gr.Button("☀️ Daily summary"),
            ]


    # Handle chat submission
    def respond(message, chat_history):
        if not message.strip():
            return "", chat_history, None

        bot_response, audio_file = gradio_chat_with_voice(message, chat_history)
        chat_history.append((message, bot_response))
        return "", chat_history, audio_file


    # Wire up events
    msg.submit(respond, [msg, chatbot], [msg, chatbot, audio_output])
    submit.click(respond, [msg, chatbot], [msg, chatbot, audio_output])
    clear.click(lambda: [], None, chatbot, queue=False)

    example_btns[0].click(lambda: "add task COMP-2540 lab due 2026-01-20", None, msg)
    example_btns[1].click(lambda: "delete task COMP-2540 lab", None, msg)
    example_btns[2].click(lambda: "what's due", None, msg)
    example_btns[3].click(lambda: "daily summary", None, msg)


print("\n🚀 Launching your AI assistant...")
demo.launch(share=True, debug=True)

/tmp/ipython-input-2819829458.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2819829458.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, bubble_full_width=False)
/tmp/ipython-input-2819829458.py:11: DeprecationWarning: The 'bubble_full_width' parameter will be removed in Gradio 6.0. This parameter no longer has any effect.
  chatbot = gr.Chatbot(height=500, bubble_full_width=False)
/tmp/ipython-input-2819829458.py:11: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False 


🚀 Launching your AI assistant...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e092957206d1b0a5a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
